In [1]:
import os
import copy
import numpy as np
import pandas as pd
import torch

from pruning import NetworkOptimization


def save_csv(arrs, pth, epoch):
    df = pd.DataFrame(np.array(arrs), columns=range(1, epoch + 1))
    if os.path.exists(pth):
        arrs_old = pd.read_csv(pth, index_col=0).values.reshape(-1, epoch)
        arrs = np.concatenate((arrs_old, np.array(arrs)), axis=0)
        df = pd.DataFrame(arrs, columns=range(1, epoch + 1))

    df.to_csv(pth)

In [2]:
epoch = 30  ## Epoch size
batch = 150 ## Batch size.
lr = 0.001 ## Learning rate.
data = 'mnist' ## Select data mnist or fashion.
edge = 5 ## Number of edge changes.
eta = 0.98 ## Decay coef.
metro = '1' 
reduce = '10,20,30,40,50,60,70,80,90,95,99,99.8'

METRO = np.array(metro.split(',')).astype(int)
REDUCE = np.array(reduce.split(',')).astype(float) * 0.01

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print('\n[v] Using device: {}'.format(device))

layer = ['fc1','fc2','fc3','fc4']


[v] Using device: cpu


In [3]:
nnOpt = copy.deepcopy(NetworkOptimization(batch_size=batch,
                                        learning_rate=lr,
                                        mask_ratio=[
                                        0, 0.0, 0.0, 0],
                                        eta=eta, data=data,
                                        edge_changed=edge))
losses, accs, epoch = nnOpt.fit(epoch_num=5,  # EPOCH
                                save_step=np.inf)
        

0.9602och: 1 | test acc: 0.96 | test loss: 0.286
0.9665och: 2 | test acc: 0.966 | test loss: 0.183
0.9707och: 3 | test acc: 0.971 | test loss: 0.143
0.9732och: 4 | test acc: 0.973 | test loss: 0.115
0.9747och: 5 | test acc: 0.975 | test loss: 0.102


# SA

In [8]:
print('[v] Start SA method.')
folder = '{}_edge{}_SA/performance_under_different_M'.format(data, edge)
print('Save to: ', folder)
if not os.path.exists(folder):
    os.makedirs(folder)
                
file_loss_untrained = 'loss_untrained.csv'
file_acc_untrained = 'acc_untrained.csv'
file_loss_trained = 'loss_trained.csv'
file_acc_trained = 'acc_trained.csv'
            
loss_untrained, acc_untrained = [], []
loss_trained, acc_trained = [], []
                
loss_untrained.append(losses[-1])
acc_untrained.append(accs[-1])
loss_trained.append(losses[-1])
acc_trained.append(accs[-1])
                
for r in REDUCE:
    print('[v] Start {:>2}% Reduced Case'.format(r * 100))
            
    nnOpt_SA = copy.deepcopy(nnOpt)
                
    loss, acc, Hq_iter = nnOpt_SA.FCSA(layer, reduce=r, metropolis=1, iteration = 200)
                
    loss_untrained.append(loss)
    acc_untrained.append(acc)

    losses_new, accs_new, epoch = nnOpt_SA.fit(epoch_num=1, save_step=np.inf)
        
    loss_trained.append(losses_new[-1])
    acc_trained.append(accs_new[-1])
                
    del nnOpt_SA

    print('[v] Finish {:>2}% Reduced Case'.format(r * 100))
    # break
        
save_csv([loss_untrained], os.path.join(folder, file_loss_untrained), epoch=len(loss_untrained))  # EPOCH
save_csv([acc_untrained], os.path.join(folder, file_acc_untrained), epoch=len(acc_untrained))  # EPOCH
save_csv([loss_trained], os.path.join(folder, file_loss_trained), epoch=len(loss_trained))  # EPOCH
save_csv([acc_trained], os.path.join(folder, file_acc_trained), epoch=len(acc_trained))  # EPOCH

print('[v] Finish SA method.')
# break

[v] Start SA method.
Save to:  mnist_edge5_SA/performance_under_different_M
[v] Start 10.0% Reduced Case
0.026572629809379578
0.9732och: 1 | test acc: 0.973 | test loss: 0.097t loss: 0.133 | T: 0.0 | M1: 90.066% | M2: 88.88% | M3: 89.84% | M4: 91.2% 
[v] Finish 10.0% Reduced Case
[v] Start 20.0% Reduced Case


KeyboardInterrupt: 

In [4]:
print('[v] Start SQA method.')
for m in METRO:
    folder = '{}_edge{}_SQA/performance_under_different_M/metropolis_{}'.format(data, edge, m)
    print('Save to: ', folder)
    if not os.path.exists(folder):
        os.makedirs(folder)
        
    print('[v] Start {} metro.'.format(m))
        
    file_loss_untrained = 'loss_untrained.csv'
    file_acc_untrained = 'acc_untrained.csv'
    file_loss_trained = 'loss_trained.csv'
    file_acc_trained = 'acc_trained.csv'
            
    loss_untrained, acc_untrained = [], []
    loss_trained, acc_trained = [], []
        
    loss_untrained.append(losses[-1])
    acc_untrained.append(accs[-1])
    loss_trained.append(losses[-1])
    acc_trained.append(accs[-1])
                
    for r in REDUCE:
        print('[v] Start {:>2}% Reduced Case'.format(r * 100))
            
        nnOpt_SQA = copy.deepcopy(nnOpt)
                
        loss, acc, Hq_iter, data_iter = nnOpt_SQA.FCSQA(layer, reduce=r, metropolis=m, iteration=200)
                
        loss_untrained.append(loss)
        acc_untrained.append(acc)

        losses_new, accs_new, epoch = nnOpt_SQA.fit(epoch_num=1, save_step=np.inf)
            
        loss_trained.append(losses_new[-1])
        acc_trained.append(accs_new[-1])
                
        del nnOpt_SQA

        print('[v] Finish {:>2}% Reduced Case'.format(r * 100))
        # break
        
    save_csv([loss_untrained], os.path.join(folder, file_loss_untrained), epoch=len(loss_untrained))  # EPOCH
    save_csv([acc_untrained], os.path.join(folder, file_acc_untrained), epoch=len(acc_untrained))  # EPOCH
    save_csv([loss_trained], os.path.join(folder, file_loss_trained), epoch=len(loss_trained))  # EPOCH
    save_csv([acc_trained], os.path.join(folder, file_acc_trained), epoch=len(acc_trained))  # EPOCH
            
    print('[v] Finish {} metro.'.format(m))
    # break
    
print('[v] Finish SQA method.')
# break

[v] Start SQA method.
Save to:  mnist_edge5_SQA/performance_under_different_M/metropolis_1
[v] Start 1 metro.
[v] Start 10.0% Reduced Case
0.028537988662719727
0.9743och: 1 | test acc: 0.974 | test loss: 0.09655 | M1: 89.982% | M2: 89.4% | M3: 90.36% | M4: 92.6%
[v] Finish 10.0% Reduced Case
[v] Start 20.0% Reduced Case


KeyboardInterrupt: 

In [5]:
print('[v] Start minK method.')
folder = '{}_minK/performance_under_different_M'.format(data)
print('Save to: ', folder)
if not os.path.exists(folder):
    os.makedirs(folder)
        
file_loss_untrained = 'loss_untrained.csv'
file_acc_untrained = 'acc_untrained.csv'
file_loss_trained = 'loss_trained.csv'
file_acc_trained = 'acc_trained.csv'
            
loss_untrained, acc_untrained = [], []
loss_trained, acc_trained = [], []

loss_untrained.append(losses[-1])
acc_untrained.append(accs[-1])
loss_trained.append(losses[-1])
acc_trained.append(accs[-1])
                
for r in REDUCE:
    print('[v] Start {:>2}% Reduced Case'.format(r * 100))
            
    nnOpt_minK = copy.deepcopy(nnOpt)
                
    loss, acc = nnOpt_minK.MINK(layer, reduce=r)
                
    loss_untrained.append(loss)
    acc_untrained.append(acc)

    losses_new, accs_new, epoch = nnOpt_minK.fit(epoch_num=1,  # epoch
                                                            save_step=np.inf)
    loss_trained.append(losses_new[-1])
    acc_trained.append(accs_new[-1])

    del nnOpt_minK

    print('[v] Finish {:>2}% Reduced Case'.format(r * 100))
    # break

        
save_csv([loss_untrained], os.path.join(folder, file_loss_untrained), epoch=len(loss_untrained))  # EPOCH
save_csv([acc_untrained], os.path.join(folder, file_acc_untrained), epoch=len(acc_untrained))  # EPOCH
save_csv([loss_trained], os.path.join(folder, file_loss_trained), epoch=len(loss_trained))  # EPOCH
save_csv([acc_trained], os.path.join(folder, file_acc_trained), epoch=len(acc_trained))  # EPOCH
            
print('[v] Finish minK method.')

[v] Start minK method.
Save to:  mnist_minK/performance_under_different_M
[v] Start 10.0% Reduced Case
0.9758och: 1 | test acc: 0.976 | test loss: 0.09691% | M2: 95.76% | M3: 95.68% | M4: 96.2% 
[v] Finish 10.0% Reduced Case
[v] Start 20.0% Reduced Case
0.9754och: 1 | test acc: 0.975 | test loss: 0.09727% | M2: 91.92% | M3: 91.68% | M4: 93.2% 
[v] Finish 20.0% Reduced Case
[v] Start 30.0% Reduced Case
0.9757och: 1 | test acc: 0.976 | test loss: 0.09305% | M2: 87.72% | M3: 87.0% | M4: 84.2% 
[v] Finish 30.0% Reduced Case
[v] Start 40.0% Reduced Case
0.978poch: 1 | test acc: 0.978 | test loss: 0.08911% | M2: 83.16% | M3: 82.96% | M4: 79.4% 
[v] Finish 40.0% Reduced Case
[v] Start 50.0% Reduced Case


KeyboardInterrupt: 